# 第二课 词向量

第二课学习目标
- 学习词向量的概念
- 用Skip-thought模型训练词向量
- 学习使用PyTorch dataset和dataloader
- 学习定义PyTorch模型
- 学习torch.nn中常见的Module
    - Embedding
- 学习常见的PyTorch operations
    - bmm
    - logsigmoid
- 保存和读取PyTorch模型

在这一份notebook中，我们会（尽可能）尝试复现论文[Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)中训练词向量的方法. 

我们会实现[Skip-gram](https://blog.csdn.net/u010665216/article/details/78721354)模型，并且使用论文中noice contrastive sampling的目标函数。

![skip_gram](skip_gram.jpg)

这篇论文有很多模型实现的细节，这些细节对于词向量的好坏至关重要。我们虽然无法完全复现论文中的实验结果，主要是由于计算资源等各种细节原因，但是我们还是可以大致展示如何训练词向量。

以下是一些我们没有实现的细节
- subsampling：参考论文section 2.3

## 调用PyTorch常用的包

In [2]:
#基本上所有torch脚本都需要用到
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud #Pytorch读取训练集需要用到torch.utils.data类

torch.nn中大多数layer在torch.nn.funtional中都有一个与之对应的函数。  
二者的[区别](https://blog.csdn.net/hawkcici160/article/details/80140059)在于：  
- torch. nn.Module中实现layer的都是一个特殊的类 会自动提取可学习的参数  
- nn.functional中的函数，更像是纯函数，由def function( )定义，只是进行简单的 数学运算而已。functional中的函数是一个确定的不变的运算公式

## 调用其他需要的包

In [3]:
from collections import Counter
import numpy as np
import random
import math

import pandas as pd
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

## 其他初始设置

In [4]:
#调用gpu
USE_CUDA=torch.cuda.is_available()

#为保证实验结果可以浮现，将各种random seed固定到一个特定的值
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)
    
#设定一些hyper parameters
C=3 #nearby words threshold 指定前后3个单词进行预测
K=100 #number of negative samples 负样本随机采样数量；每一个正样本对应K个负样本
NUM_EPOCHS=2 #The num of epochs of training 迭代轮数
MAX_VOCAB_SIZE=30000 #the vocabulary size 词汇表大小
BATCH_SIZE=128
LEARNING_RATE=0.2 #the initial learning rate
EMBEDDING_SIZE=100 #词向量维度

#tokenize函数 将文本转化为一个个单词
def word_tokenize(text):
    return text.split()

## 数据预处理及相关操作
- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示除MAX_VOCAB_SIZE个单词外其他所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [5]:
#读取文件
with open('./text8/text8.train.txt','r') as fi:
    text=fi.read()
    
# len(text)

#分词
#str.lower()将str中大写转化为小写
text=[w for w in word_tokenize(text.lower())]

#将出现频率最高的MAX_VOCAB_SIZE-1个单词取出来，以字典的形式存储(包含每个单词出现次数)
#-1留给UNK单词
#collection.Counter(text): 计算每个元素出现个数 返回counter对象
#Counter(text).most_common(N): 找到text中出现最多的前N个元素
#https://zhuanlan.zhihu.com/p/350899229
vocab=dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))
#将UNK单词添加进vocab
#UNK出现次数=总单词出现次数-常见单词出现次数
#dic.values() 返回字典中所有值所构成的对象
vocab['<unk>']=len(text)-np.sum(list(vocab.values()))

#从vocab中取出所有单词
idx_to_word=[word for word in vocab.keys()]

#以字典的形式取得单词及其对应的索引
#enumerate: 接收一个可遍历的数据对象['a','b','c'] 返回索引与对象的组合[(0,'a'),(1,'b'),(2,'c')]
#索引值与单词出现次数相反，最常见单词索引为0。
word_to_idx={word:i for i,word in enumerate(idx_to_word)}

# list(word_to_idx.items())[:100]

#计算每个单词频率 负采样时需要使用
#获得所有单词出现的次数
word_counts=np.array([count for count in vocab.values()], dtype=np.float32)
#计算所有单词的频率
word_freqs=word_counts/np.sum(word_counts)
#论文Distributed Representations of Word...中频率取了3/4次方
word_freqs=word_freqs**(3./4.)
#重新normalize 重新计算所有单词频率 类似softmax
word_freqs=word_freqs/np.sum(word_freqs)

#检查单词数为MAX_VOCAB_SIZE
VOCAB_SIZE=len(idx_to_word)
VOCAB_SIZE

30000

## 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字，然后用subsampling预处理这些文字。
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

In [18]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        #初始化模型
        #super(WordEmbeddingDataset, self).__init__()
        super().__init__()
        
        #顺序存储每个在text中word的word_to_idx中的索引(序号)，
        #如果word不在word_to_idx中（属于unk）则存储unk在word_to_idx中对应的序号
        self.text_encoded=[word_to_idx.get(word,word_to_idx['<unk>']) for word in text]
        #转化为int型LongTensor
        self.text_encoded=torch.LongTensor(self.text_encoded)
        
        #将输入的参数初始化为torch tensor
        # self.word_to_idx=torch.Tensor(word_to_idx)
        # self.idx_to_word=torch.Tensor(idx_to_word)
        self.word_to_idx=word_to_idx
        self.idx_to_word=idx_to_word
        self.word_freqs=torch.Tensor(word_freqs)
        self.word_counts=torch.Tensor(word_counts)
        
    
    #数据集一共有多少个item
    def __len__(self):
        return len(self.tesxt_encoded)
    
    #提供一个index 返回一串训练数据
    def __getitem__(self, idx):
        ##中心词 
        #center_word=self.text_encoded[idx]
        ##print(center_word)
        ##中心词索引
        ##这里__getitem__函数是个迭代器，idx代表了所有的单词索引。
        
        #pos_indices = list(range(idx-C, idx)) + list(range(idx+1, idx+C+1))
        ##周围词索引的索引，比如idx=0时。pos_indices = [-3, -2, -1, 1, 2, 3] 
        ##老师讲这里的时候，我不是特别明白
        
        #pos_indices = [i%len(self.text_encoded) for i in pos_indices]
        ##range(idx+1, idx+C+1)超出词汇总数时，需要特别处理，取余数
        
        #pos_words = self.text_encoded[pos_indices]
        ##周围词索引，就是希望出现的正例单词
        ##print(pos_words)
        
        #neg_words = torch.multinomial(self.word_freqs, K * pos_words.shape[0], True)
        ##负例采样单词索引，torch.multinomial作用是对self.word_freqs做K * pos_words.shape[0]次取值，输出的是self.word_freqs对应的下标。
        ##取样方式采用有放回的采样，并且self.word_freqs数值越大，取样概率越大。
        ##每个正确的单词采样K个，pos_words.shape[0]是正确单词数量
        ##print(neg_words)
        #
        #return center_word, pos_words, neg_words
        
        #中心词***
        center_word=self.text_encoded[idx]
        
        #周围词
        pos_indices=list(range(idx-C))+list(range(idx+1,idx+1+C))

In [41]:
dataset=WordEmbeddingDataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)
# dte=dataset.text_encoded
# print(dte)
# print(len(dte))
# dtet=dte.tolist()
# print(dtet[:100])
# print(type(dtet[0]))
# # print(dict(Counter(dtet)))
# print([(k,v) for i, (k, v) in enumerate(dict(Counter(dtet)).items()) if i <100])

tensor([ 4813,  3139,    11,  ...,    50, 29885,   393])
15304686
[4813, 3139, 11, 5, 194, 1, 3015, 46, 59, 155, 127, 741, 461, 10485, 133, 0, 25752, 1, 0, 108, 833, 2, 0, 16267, 29999, 1, 0, 152, 833, 3493, 0, 194, 10, 186, 59, 4, 5, 10620, 213, 6, 1332, 102, 437, 19, 59, 2764, 355, 6, 3625, 0, 709, 1, 364, 26, 40, 37, 53, 527, 97, 11, 5, 1398, 2929, 18, 562, 691, 6644, 0, 252, 4813, 10, 1043, 27, 0, 316, 247, 29999, 2964, 789, 189, 4813, 11, 5, 201, 569, 10, 0, 1107, 19, 2581, 25, 8819, 2, 273, 31, 4089, 140, 58, 25, 6494]
<class 'int'>
[(4813, 303), (3139, 505), (11, 119348), (5, 293387), (194, 6533), (1, 537144), (3015, 531), (46, 25760), (59, 20455), (155, 7640), (127, 9129), (741, 2054), (461, 3111), (10485, 106), (133, 8694), (0, 959616), (25752, 25), (108, 10582), (833, 1862), (2, 376233), (16267, 54), (29999, 617240), (152, 7750), (3493, 442), (10, 166132), (186, 6664), (4, 335477), (10620, 104), (213, 5793), (6, 285950), (1332, 1203), (102, 10796), (437, 3216), (19, 99419), (